# Forecasting Time Series with optimization based models

The time series repository used to evaluate the models can be found here: http://sites.labic.icmc.usp.br/icmc_tspr/index.php/datasets

The preprocessing procedure was based on : https://www.researchgate.net/publication/320321525_Forecasting_Across_Time_Series_Databases_using_Long_Short-Term_Memory_Networks_on_Groups_of_Similar_Series. Items 3.4 to 3.7 with some small changes.

The models tested were: 
- Linear Support Vector Machine (https://scikit-learn.org/stable/modules/svm.html#mathematical-formulation - item 1.4.7.5)
- Bayesian Regression (https://mml-book.github.io/book/mml-book.pdf - pages 303 to 313)

Both with and without applying the STL decomposition, the other pipeline components like rolling window, and local de-trending are kept fixed.

And the baselines were:
- auto.arima
- ets

## Assumptions we'll make

- Seasonal pattern doesn't change over time;
- STL residuals are white noise;
- The time series comes from an autoregressive process.

## Assumptions we'll relax

- The training set is independent and identically distributed according to a fixed distribution $\rho$ on $\mathcal{Y}^p \times \mathcal{Y}$, where $p$ is the autoregressive order.

## Evaluation procedure

We splitted each time series using a holdout technique, 95% to train e 5% to test. The functions $\textit{ets}$, $\textit{auto.arima}$ and $\textit{forecast}$ from the $\textit{forecast}$ package were used to obtain the predictions for the baseline models. On the other hand, the svm and the bayesian regression were trained using the following:

    0. function evaluation(model:svm or bayesian, y:time series, decompose:bool):
    1.     split y into train and test
    2.     apply log to train
    3.     if decompose is True:
    4.         decompose train using STL
    5.         fit naive forecaster on seasonal component
    6.         split (trend + residual) into train* and validation
    7.         apply grid search on (train*, validation) 
    8.         forecast using model initialized with the best parameters found
    9.         exponentiate the forecasted values
    10.        return test and forecasted values
    11.   else:
    12.        split train into train* and validation
    13.        repeat lines 7. to 10.

The search space for the SVM was the same used in here: https://www.researchgate.net/publication/330742498_Evaluation_of_statistical_and_machine_learning_models_for_time_series_prediction_Identifying_the_state-of-the-art_and_the_best_conditions_for_the_use_of_each_model, page 21. Moreover, the bayesian regression is able to optimize its own parameters like is described here : https://scikit-learn.org/stable/modules/linear_model.html#bayesian-regression.

We used the mean absolute error (MAE) to compare the models, now we'll analyse the results

## Exploratory Data Analysis

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("results.csv", index_col=0)
df.head()

,category,autoets,autoarima,linear_svr,stl_linear_svr,bayesian_ridge,stl_bayesian_ridge,residuals_are_independent
0,deterministic,73.896820,9.345932,0.118581,1.708678e-12,0.000034,1.708678e-12,False
1,deterministic,73.896820,49.189988,0.203931,1.969832e-01,0.053057,1.541884e-01,False
2,deterministic,73.896820,48.952778,2.767953,4.987801e-01,3.314283,4.937540e-01,False
3,deterministic,7.331475,3.425046,0.625135,4.626489e+00,0.616375,4.694656e+00,False
4,deterministic,6.647874,7.445414,0.296796,4.533904e+00,0.556557,4.600456e+00,False


The column category tells us the type of the time series considered, the columns 2 to 7 are the MAE's of each model and the last column indicates wheter the residuals from the STL decomposition are white noise using the Ljung-Box test: https://www.statsmodels.org/stable/generated/statsmodels.stats.diagnostic.acorr_ljungbox.html

In order to make the analysis easier, we'll rank the models from best (column 1) to worst (column 6)

In [11]:
def rank(df:pd.DataFrame)->pd.DataFrame:
    rank_row = lambda row:pd.Series.sort_values(row[1:-1]).index.tolist()
    return pd.DataFrame(
        [rank_row(row)+[row[-1]] for i, row in df.iterrows()], 
        columns=["1", "2", "3", "4", "5", "6", "residuals_are_independent"])

### Deterministic data

In [12]:
rank(df[df.category=="deterministic"])

,1,2,3,4,5,6,residuals_are_independent
0,stl_linear_svr,stl_bayesian_ridge,bayesian_ridge,linear_svr,autoarima,autoets,False
1,bayesian_ridge,stl_bayesian_ridge,stl_linear_svr,linear_svr,autoarima,autoets,False
2,stl_bayesian_ridge,stl_linear_svr,linear_svr,bayesian_ridge,autoarima,autoets,False
3,bayesian_ridge,linear_svr,autoarima,stl_linear_svr,stl_bayesian_ridge,autoets,False
4,linear_svr,bayesian_ridge,stl_linear_svr,stl_bayesian_ridge,autoets,autoarima,False
5,linear_svr,bayesian_ridge,stl_linear_svr,stl_bayesian_ridge,autoets,autoarima,False
6,bayesian_ridge,linear_svr,stl_bayesian_ridge,autoets,autoarima,stl_linear_svr,False
7,bayesian_ridge,linear_svr,autoarima,stl_bayesian_ridge,autoets,stl_linear_svr,False
8,linear_svr,bayesian_ridge,autoarima,stl_bayesian_ridge,stl_linear_svr,autoets,False
9,bayesian_ridge,linear_svr,autoarima,stl_bayesian_ridge,stl_linear_svr,autoets,False


When it comes to deterministic time series the machine learning models tend to outperform both the baseline and the STL framework, it's worth to notice the residuals are not independent in almost all datasets.

### Stochastic data

In [16]:
rank(df[df.category=="stochastic"])

,1,2,3,4,5,6,residuals_are_independent
0,autoarima,autoets,stl_linear_svr,stl_bayesian_ridge,bayesian_ridge,linear_svr,True
1,bayesian_ridge,linear_svr,autoarima,stl_bayesian_ridge,stl_linear_svr,autoets,True
2,autoets,linear_svr,bayesian_ridge,stl_bayesian_ridge,stl_linear_svr,autoarima,False
3,autoets,linear_svr,bayesian_ridge,stl_linear_svr,stl_bayesian_ridge,autoarima,False
4,stl_linear_svr,autoarima,stl_bayesian_ridge,autoets,bayesian_ridge,linear_svr,True
5,autoarima,autoets,stl_linear_svr,stl_bayesian_ridge,bayesian_ridge,linear_svr,True
6,linear_svr,bayesian_ridge,stl_linear_svr,stl_bayesian_ridge,autoarima,autoets,True
7,autoarima,linear_svr,bayesian_ridge,stl_bayesian_ridge,stl_linear_svr,autoets,True
8,linear_svr,bayesian_ridge,stl_linear_svr,stl_bayesian_ridge,autoarima,autoets,True
9,linear_svr,bayesian_ridge,stl_linear_svr,stl_bayesian_ridge,autoarima,autoets,True


Here, although the majority of the time series could have had its components decomposed successfully, the STL failed.

### Chaotic data

In [17]:
rank(df[df.category=="chaotic"])

,1,2,3,4,5,6,residuals_are_independent
0,linear_svr,bayesian_ridge,stl_bayesian_ridge,stl_linear_svr,autoets,autoarima,False
1,autoets,autoarima,bayesian_ridge,linear_svr,stl_bayesian_ridge,stl_linear_svr,False
2,autoarima,stl_linear_svr,stl_bayesian_ridge,linear_svr,bayesian_ridge,autoets,False
3,linear_svr,bayesian_ridge,autoarima,stl_bayesian_ridge,stl_linear_svr,autoets,False
4,autoarima,bayesian_ridge,linear_svr,stl_linear_svr,stl_bayesian_ridge,autoets,False
5,bayesian_ridge,autoets,linear_svr,stl_bayesian_ridge,stl_linear_svr,autoarima,False
6,autoarima,linear_svr,bayesian_ridge,autoets,stl_linear_svr,stl_bayesian_ridge,False
7,linear_svr,bayesian_ridge,autoarima,stl_bayesian_ridge,stl_linear_svr,autoets,False


We can spot no pattern at all, the STL decomposition is not trustworthy.

### Daily

In [18]:
rank(df[df.category=="daily"])

,1,2,3,4,5,6,residuals_are_independent
0,stl_linear_svr,stl_bayesian_ridge,bayesian_ridge,linear_svr,autoets,autoarima,False
1,autoarima,bayesian_ridge,linear_svr,autoets,stl_bayesian_ridge,stl_linear_svr,False
2,autoets,autoarima,bayesian_ridge,linear_svr,stl_bayesian_ridge,stl_linear_svr,False
3,autoarima,autoets,stl_linear_svr,stl_bayesian_ridge,linear_svr,bayesian_ridge,False
4,autoets,autoarima,stl_linear_svr,stl_bayesian_ridge,bayesian_ridge,linear_svr,False
5,stl_bayesian_ridge,stl_linear_svr,autoets,autoarima,bayesian_ridge,linear_svr,True
6,autoets,autoarima,stl_linear_svr,stl_bayesian_ridge,bayesian_ridge,linear_svr,False
7,autoarima,stl_bayesian_ridge,stl_linear_svr,bayesian_ridge,linear_svr,autoets,False
8,autoarima,stl_bayesian_ridge,stl_linear_svr,autoets,linear_svr,bayesian_ridge,False
9,autoarima,bayesian_ridge,linear_svr,stl_bayesian_ridge,stl_linear_svr,autoets,False


The STL wasn't able to decompose the daily time series, probably due to the presence of multiple hidden patterns. Thus, we cannot guarantee that our framework can be applied successfully.

### Monthly data

In [19]:
rank(df[df.category=="monthly"])

,1,2,3,4,5,6,residuals_are_independent
0,stl_linear_svr,stl_bayesian_ridge,autoets,autoarima,linear_svr,bayesian_ridge,True
1,autoets,stl_linear_svr,stl_bayesian_ridge,autoarima,bayesian_ridge,linear_svr,True
2,autoets,stl_bayesian_ridge,stl_linear_svr,autoarima,bayesian_ridge,linear_svr,False
3,autoets,stl_bayesian_ridge,stl_linear_svr,bayesian_ridge,linear_svr,autoarima,True
4,autoarima,autoets,stl_linear_svr,stl_bayesian_ridge,linear_svr,bayesian_ridge,False
5,autoets,linear_svr,autoarima,bayesian_ridge,stl_linear_svr,stl_bayesian_ridge,False
6,autoets,stl_linear_svr,stl_bayesian_ridge,linear_svr,bayesian_ridge,autoarima,False
7,autoets,stl_linear_svr,stl_bayesian_ridge,linear_svr,bayesian_ridge,autoarima,False
8,autoets,bayesian_ridge,linear_svr,autoarima,stl_linear_svr,stl_bayesian_ridge,False
9,autoarima,autoets,bayesian_ridge,linear_svr,stl_linear_svr,stl_bayesian_ridge,False


Looking at the datasets in which the residuals are white noise we can spot that: in 10 out of 15 samples the STL-SVM had a lower error in comparison with the raw SVM estimator and moreover, in 11 out 15 the STL-BAYESIAN also was better than the BAYESIAN. Finally, we can see that the STL framework, even improving the accuracy, it's not enough to be better than the exponential smoothing and arima models.

### Other

In [29]:
rank(pd.concat([df[df.category=="annual"], df.iloc[-4:,:]]))

,1,2,3,4,5,6,residuals_are_independent
0,autoets,autoarima,linear_svr,bayesian_ridge,stl_linear_svr,stl_bayesian_ridge,True
1,linear_svr,bayesian_ridge,autoarima,stl_linear_svr,stl_bayesian_ridge,autoets,False
2,linear_svr,stl_linear_svr,bayesian_ridge,stl_bayesian_ridge,autoarima,autoets,NaN
3,stl_linear_svr,stl_bayesian_ridge,autoarima,autoets,bayesian_ridge,linear_svr,True
4,autoets,bayesian_ridge,linear_svr,autoarima,stl_bayesian_ridge,stl_linear_svr,False
5,autoarima,linear_svr,bayesian_ridge,autoets,stl_bayesian_ridge,stl_linear_svr,False
6,autoarima,bayesian_ridge,linear_svr,autoets,stl_linear_svr,stl_bayesian_ridge,False


Not enough data to conclude something.